In [1]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pprint import pprint
import math
from scipy.io import arff


In [2]:
# params
seed = 42

Датасет включает в себя данные отчетности банков РФ с 2008 года. Изначально имеется 5 таблиц: 
1. PNL_aggregated.h5, BS_aggregated.h5 содержат группированную отчетность вплоть до самого нижнего уровня группировки (что порождает сотни столбцов). Важно: в этих таблицах имеются столбцы для всех уровней группировки, а значит для каждого банка есть значение И активов, И, к примеру, инвестиций входящих в эти активы.
2. Derevya.xlsx содержит схему группировки для всех периодов для разных уровней (файл для ознакомления, группировка на его основе уже выполнена)
3. BankDefaults.xlsx содержит список дефолтнувших банков и даты дефолтов
4. json-файлы, содержащие словари, дающие списки названий переменных для каждого из уровней дерева: names_levels_bs.json и names_levels_pnl.json


Загрузим данные

In [3]:
df_pnl = pd.read_hdf("data//PNL_aggregated.h5", key = "102")
df_bs = pd.read_hdf("data//BS_aggregated.h5", key = "102")


Загрузим словари с уровнями переменных

In [4]:
with open('data//names_levels_pnl.json', encoding="utf-8") as f:
    names_levels_pnl = json.load(f)
with open('data//names_levels_bs.json', encoding="utf-8") as f:
    names_levels_bs = json.load(f)

In [5]:
df_pnl[names_levels_pnl["2"] + ["DT", "REGN"]]

,Прибыль /(убыток) до налогообложения,Расходы по налогу на прибыль,DT,REGN
0,-5439182.0,NaN,2007-01-01,1
1,-13587.0,NaN,2007-01-01,3
2,-15687.0,NaN,2007-01-01,21
3,-23626.0,NaN,2007-01-01,52
4,-39139.0,NaN,2007-01-01,55
...,...,...,...,...
42130,42432.0,1523.0,2021-10-01,3533
42131,16691.0,-250.0,2021-10-01,3536
42132,-58247.0,6409.0,2021-10-01,3538
42133,-1524203.0,316959.0,2021-10-01,3539


Загрузим данные по дефолтам

In [126]:
df_defaults = pd.read_excel("data//BankDefaults.xlsx")
df_defaults = df_defaults[["BankDefaultIndex", "Name", "regnum", "DefaultType", "DefaultDate","BankLocalization"]]
df_defaults = df_defaults.rename(columns = {"regnum" : "REGN"})
df_defaults

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,BankDefaultIndex,Name,REGN,DefaultType,DefaultDate,BankLocalization
0,1,Финчер,3486-К,ликв.,24.09.2021,Москва
1,2,Платина,2347,отозв.,17.09.2021,Москва
2,3,КС Банк,1752,отозв.,06.08.2021,Саранск (Республика Мордовия)
3,4,Руна-Банк,3207,отозв.,23.07.2021,Москва
4,5,Русское Финансовое Общество,3427-К,отозв.,23.07.2021,Москва
...,...,...,...,...,...,...
2784,2785,Сасовобанк,862,ликв.,28.12.1991,"Сасово, Рязанская обл."
2785,2786,Михайловский,893,ликв.,28.12.1991,Unknown
2786,2787,Мариинско-Посадский Коммерческий Банк,1021,ликв.,28.12.1991,"Мариинский Посад, Чувашия"
2787,2788,"Конверсия, Реконструкция и Развитие",472,отозв.,11.07.1991,Москва


Напишем функцию, которая будет брать нужные нам уровни в обоих таблицах и мёрджить их

In [292]:
class create_name_masks_container():
    
    def __init__(self, names_levels_pnl, names_levels_bs, pnl, bs, pnl_level, bs_level):
    #currently not all the names in the dictionary are real columns
    #luckily, not much of them
    #if I will ever fix this bug, I will deprecate the code below
        if (hasattr(pnl_level, '__len__') == False):
            self.pnl_mask = [i for i in np.array(names_levels_pnl[str(pnl_level)]) if i in np.array(df_pnl.columns)]
        else: 
            self.pnl_mask = []
            for level in pnl_level:
                self.pnl_mask = self.pnl_mask + [i for i in np.array(names_levels_pnl[str(level)]) if i in np.array(df_pnl.columns)]

        if (hasattr(bs_level, '__len__') == False):
            self.bs_mask = [i for i in np.array(names_levels_bs[str(bs_level)]) if i in np.array(df_bs.columns)]
        else: 
            self.bs_mask = []
            for level in bs_level:
                self.bs_mask = self.bs_mask + [i for i in np.array(names_levels_bs[str(level)]) if i in np.array(df_bs.columns)]

        self.pnl_encoding = [f"PNL{str(i)}" for i in range(1, len(self.pnl_mask) + 1)]
        self.bs_encoding = [f"BS{str(i)}" for i in range(1, len(self.bs_mask) + 1)]
        
    
    def full_true_mask(self):
        return ["DT", "REGN"] + self.pnl_mask + self.bs_mask
    
    def pnl_index_mask(self):
        return ["DT", "REGN"] + self.pnl_mask
    
    def bs_index_mask(self):
        return ["DT", "REGN"] + self.bs_mask
    
    def encoding_mask(self):
        return ["DT", "REGN"] + self.pnl_encoding + self.bs_encoding
    
class create_days_container():
    
    def __init__(self, target_days):
        
        if (hasattr(target_days, '__len__') == False):
            self.target_days = [target_days]
        else:
            self.target_days = target_days
        
        self.target_names = [""]*len(self.target_days)
        for days_index in range(len(self.target_days)):
            self.target_names[days_index] = f"DefaultIn{self.target_days[days_index]}Days"
        
    def compare_days(self, number_of_days_real, number_of_days_benchmark):
        if pd.isnull(number_of_days_real):
            return 0
        else:
            return (int(number_of_days_real.days) <= int(number_of_days_benchmark))*1
        
    def create_target_columns(self, df, days_to_default_column = "DaysToDefault"):
        days_to_default_column = df[days_to_default_column]

        for days, days_name in zip(self.target_days, self.target_names):
            df[days_name] = days_to_default_column.apply(self.compare_days, number_of_days_benchmark = days)
        return df
    
    
def prepare_df_to_modelling(pnl, bs, defaults, name_masks, target_days):

    #create bs and pnl of needed level
    restricted_pnl = pnl[name_masks_container.pnl_index_mask()]
    restricted_bs = bs[name_masks_container.bs_index_mask()]
    
    #merge bs and pnl
    merged_reporting = restricted_bs.merge(restricted_pnl, on = ["DT", "REGN"])
    merged_reporting = merged_reporting[name_masks_container.full_true_mask()]
    merged_reporting.columns = name_masks_container.encoding_mask()
    
    #merge with defaults
    def func2(x):
        if type(x) == type("str"):
            return datetime.datetime.strptime(str(x), "%d.%m.%Y") 
        else:
            return x
        
    merged_reporting.REGN = merged_reporting.REGN.apply(str)
    pd.options.mode.chained_assignment = None
    defaults = defaults[defaults.DefaultType != "ликв."]
    defaults.DefaultDate = defaults.DefaultDate.apply(func2)
    merged_reporting = merged_reporting.merge(defaults, how = "left", on = "REGN")
    merged_reporting["DaysToDefault"] = merged_reporting.DefaultDate - merged_reporting.DT
    days_container_instance = create_days_container(target_days)

    merged_reporting = days_container_instance.create_target_columns(merged_reporting)
    
    merged_reporting.drop(list(defaults.columns) + ["DaysToDefault"], axis = 1, inplace = True)
    
    merged_reporting["Year"] = merged_reporting.DT.apply(lambda x: x.year)
    merged_reporting["Month"] = merged_reporting.DT.apply(lambda x: x.month)

    return merged_reporting


In [303]:
name_masks_container = create_name_masks_container(names_levels_pnl, names_levels_bs, df_pnl, df_bs, [1, 5], [1, 2])
df = prepare_df_to_modelling(df_pnl, df_bs, df_defaults, name_masks, [365, 365*2, 10000])
df

,DT,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,PNL9,...,BS25,BS26,BS27,BS28,BS29,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month
0,2007-01-01,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,-6276.0,...,-45907.0,-9819030.0,-7819685.0,-7837.0,-12854441.0,0,0,0,2007,1
1,2007-01-01,-13587.0,-38732.0,2553.0,-10168.0,-3369.0,-14695.0,0.0,-85.0,-662.0,...,-66.0,-97.0,-41775.0,-60353.0,-24950.0,0,1,1,2007,1
2,2007-01-01,-15687.0,-54966.0,11502.0,-753.0,-18148.0,-26699.0,-4.0,NaN,-900.0,...,-68.0,-231.0,-87419.0,NaN,-34275.0,0,0,0,2007,1
3,2007-01-01,-23626.0,-82355.0,8694.0,-1966.0,-17985.0,-58636.0,NaN,NaN,-3678.0,...,-307.0,-6247.0,-39060.0,-75576.0,-57626.0,0,0,1,2007,1
4,2007-01-01,-39139.0,-144480.0,-25297.0,-5440.0,-17760.0,-73336.0,-140.0,-1632.0,353.0,...,-540.0,-1695.0,-82500.0,-60000.0,-43170.0,0,0,1,2007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42171,2021-10-01,43955.0,-2566.0,-2895.0,0.0,-405.0,-10176.0,0.0,-622.0,44.0,...,0.0,-6202.5,-90000.0,-180660.0,169681.0,0,0,0,2021,10
42172,2021-10-01,16441.0,-39063.0,0.0,0.0,0.0,-50251.0,0.0,1053.0,23.0,...,-780.0,-20316.5,-90000.0,-233383.0,212976.0,0,0,0,2021,10
42173,2021-10-01,-51838.0,-35672.0,0.0,0.0,-12556.0,-104025.0,0.0,1506.0,8505.0,...,-1707.0,-56727.0,-340000.0,-91041.0,-51073.0,0,0,0,2021,10
42174,2021-10-01,-1207244.0,-63160.0,3771.0,-18492.0,20289.0,-1556192.0,0.0,3748.0,-80960.0,...,-4559.0,-351335.0,-702495.0,NaN,-2081503.0,0,0,0,2021,10


### Список факторов:


In [308]:
factors_dict = pd.DataFrame({"Name" : name_masks_container.full_true_mask() + ["Year", "Month"], 
              "Attribute" : name_masks_container.encoding_mask()+ ["Year", "Month"]})
factors_dict

,Name,Attribute
0,DT,DT
1,REGN,REGN
2,Чистая прибыль /(убыток),PNL1
3,Чистый процент.доход/(убыток) до формирования/...,PNL2
4,Возмещение /(формирование) резервов на потери ...,PNL3
5,Чистые непроцентные доходы по операциям с ценн...,PNL4
6,"Чистые непроцентные доходы по операциям с ПФИ,...",PNL5
7,Чистые комиссионные доходы,PNL6
8,Чистые доходы от инвестиций,PNL7
9,Чистый доход по операциям сдечи имущества в ар...,PNL8


Для единообразия нотации, создадим новую зависимую переменную target:

In [305]:
df.rename(columns = {"class":"target"}, inplace = True)

Пропуски заполнены NaN:

In [307]:
df[df.PNL3.isnull()]

,DT,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,PNL9,...,BS25,BS26,BS27,BS28,BS29,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month
371,2007-01-01,-1271.0,-8348.0,NaN,NaN,NaN,-3980.0,NaN,NaN,-65.0,...,-136.0,-1.5,-5000.0,NaN,-108174.0,0,0,0,2007,1
496,2007-04-01,-51.0,-185.0,NaN,NaN,NaN,-125.0,NaN,NaN,1.0,...,0.0,-1.5,-4315.0,NaN,-841.0,0,0,1,2007,4
569,2007-04-01,-1234.0,-1916.0,NaN,NaN,NaN,-1511.0,NaN,NaN,-1.0,...,-7.0,-3.0,-36000.0,NaN,-5253.0,0,0,1,2007,4
758,2007-04-01,-272.0,NaN,NaN,NaN,NaN,-476.0,NaN,NaN,NaN,...,0.0,0.0,-820.0,NaN,-2697.0,0,0,0,2007,4
1004,2007-04-01,-708.0,-2439.0,NaN,NaN,NaN,-1035.0,NaN,NaN,NaN,...,-34.0,-110.0,-5000.0,NaN,-71338.0,0,0,0,2007,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39799,2020-04-01,-4577.0,-480.0,NaN,92.0,-171.0,-39086.0,0.0,509.0,4663.0,...,-1332.0,-23336.5,-28000.0,-30000.0,-36860.0,0,0,0,2020,4
40216,2020-07-01,-5588.0,-1364.0,NaN,-75.0,-164.0,-76673.0,0.0,742.0,8485.0,...,-1117.0,-20055.5,-28000.0,-30000.0,-37762.0,0,0,0,2020,7
40626,2020-10-01,-6233.0,-2050.0,NaN,-31.0,-2146.0,-109555.0,0.0,871.0,8056.0,...,-1293.0,-42455.0,-28000.0,-45000.0,-38407.0,0,0,0,2020,10
41022,2021-01-01,-45423.0,-3371.0,NaN,-18.0,-4799.0,-173371.0,0.0,1591.0,7462.0,...,-1115.0,-40826.5,-28000.0,-30000.0,-77597.0,0,0,0,2021,1


dfВыделяем test&train сеты

In [310]:
df

,DT,PNL1,PNL2,PNL3,PNL4,PNL5,PNL6,PNL7,PNL8,PNL9,...,BS25,BS26,BS27,BS28,BS29,DefaultIn365Days,DefaultIn730Days,DefaultIn10000Days,Year,Month
0,2007-01-01,-5439182.0,-7554951.0,1811692.0,194368.0,-1419535.0,-2369408.0,-14492.0,-4699.0,-6276.0,...,-45907.0,-9819030.0,-7819685.0,-7837.0,-12854441.0,0,0,0,2007,1
1,2007-01-01,-13587.0,-38732.0,2553.0,-10168.0,-3369.0,-14695.0,0.0,-85.0,-662.0,...,-66.0,-97.0,-41775.0,-60353.0,-24950.0,0,1,1,2007,1
2,2007-01-01,-15687.0,-54966.0,11502.0,-753.0,-18148.0,-26699.0,-4.0,NaN,-900.0,...,-68.0,-231.0,-87419.0,NaN,-34275.0,0,0,0,2007,1
3,2007-01-01,-23626.0,-82355.0,8694.0,-1966.0,-17985.0,-58636.0,NaN,NaN,-3678.0,...,-307.0,-6247.0,-39060.0,-75576.0,-57626.0,0,0,1,2007,1
4,2007-01-01,-39139.0,-144480.0,-25297.0,-5440.0,-17760.0,-73336.0,-140.0,-1632.0,353.0,...,-540.0,-1695.0,-82500.0,-60000.0,-43170.0,0,0,1,2007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42171,2021-10-01,43955.0,-2566.0,-2895.0,0.0,-405.0,-10176.0,0.0,-622.0,44.0,...,0.0,-6202.5,-90000.0,-180660.0,169681.0,0,0,0,2021,10
42172,2021-10-01,16441.0,-39063.0,0.0,0.0,0.0,-50251.0,0.0,1053.0,23.0,...,-780.0,-20316.5,-90000.0,-233383.0,212976.0,0,0,0,2021,10
42173,2021-10-01,-51838.0,-35672.0,0.0,0.0,-12556.0,-104025.0,0.0,1506.0,8505.0,...,-1707.0,-56727.0,-340000.0,-91041.0,-51073.0,0,0,0,2021,10
42174,2021-10-01,-1207244.0,-63160.0,3771.0,-18492.0,20289.0,-1556192.0,0.0,3748.0,-80960.0,...,-4559.0,-351335.0,-702495.0,NaN,-2081503.0,0,0,0,2021,10


In [312]:
X = df.drop(["DefaultIn365Days", "DefaultIn730Days", "DefaultIn10000Days"], axis = 1)
y = df[["DefaultIn365Days", "DefaultIn730Days", "DefaultIn10000Days"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


Сохраняем данные

In [ ]:
!pip install pyarrow
!pip install fastparquet

In [313]:
# Save data & info ===
# parquet is optimized for large volumes of data
!mkdir samples
X_train.to_parquet('./samples/X_train.parquet')
X_test.to_parquet('./samples/X_test.parquet')
# переводим pd.Series в pd.DataFrame для удобного экспорта
pd.DataFrame(y_train).to_parquet('./samples/y_train.parquet')
pd.DataFrame(y_test).to_parquet('./samples/y_test.parquet')

#списки категориальных и количественных переменных
df_number_of_uniques = df.nunique()
presumably_continuous = df_number_of_uniques[df_number_of_uniques >= 15]
presumably_discrete = df_number_of_uniques[df_number_of_uniques < 15]

presumably_continuous_names = list(presumably_continuous.index)
presumably_discrete_names = list(presumably_discrete.index)

with open('factors.json', 'w') as f:
    json.dump({'cat_vals': presumably_discrete_names, "num_vals": presumably_continuous_names}, f)

### Статистики

Целевых событий на горизонте года немного, на горизонте двух лет - достаточно прилично, на всем периоде наблюдения - почти половина

In [316]:
print(f'Количество наблюдений: {X.shape[0]}')
print(f'Количество наблюдений, где имеются данные о дефолте или его отсутствии: {X[y.isnull() == False].shape[0]}')
print(f'Количество факторов: {X.shape[1]}')
print(f'Количество целевых событий: {y.sum()}')
print(f'Доля целевых событий: {y.sum() / X[y.isnull() == False].shape[0] * 100}%')

Количество наблюдений: 42176
Количество наблюдений, где имеются данные о дефолте или его отсутствии: 42176
Количество факторов: 45
Количество целевых событий: DefaultIn365Days       2029
DefaultIn730Days       4139
DefaultIn10000Days    16212
dtype: int64
Доля целевых событий: DefaultIn365Days       4.810793
DefaultIn730Days       9.813638
DefaultIn10000Days    38.438923
dtype: float64%


### Пропуски

В датасете есть переменные почти без пропусков, есть с большим количеством. Пропуски есть смысл заполнить нулями, потому что они практически всегда означают, что банк не отчитался о той или иной строке отчетности.

In [317]:
for col in X.columns:
    print(f'Количество пропусков по фактору {col}:\n\t{X[col].isna().sum()} или {X[col].isna().sum() / df.shape[0] * 100}%')

Количество пропусков по фактору DT:
	0 или 0.0%
Количество пропусков по фактору PNL1:
	0 или 0.0%
Количество пропусков по фактору PNL2:
	129 или 0.30586115326251895%
Количество пропусков по фактору PNL3:
	126 или 0.29874810318664646%
Количество пропусков по фактору PNL4:
	3043 или 7.215003793626708%
Количество пропусков по фактору PNL5:
	870 или 2.062784522003035%
Количество пропусков по фактору PNL6:
	6 или 0.014226100151745068%
Количество пропусков по фактору PNL7:
	4904 или 11.627465857359635%
Количество пропусков по фактору PNL8:
	1935 или 4.587917298937785%
Количество пропусков по фактору PNL9:
	93 или 0.22050455235204858%
Количество пропусков по фактору PNL10:
	28148 или 66.73937784522003%
Количество пропусков по фактору PNL11:
	0 или 0.0%
Количество пропусков по фактору PNL12:
	17234 или 40.86210166919575%
Количество пропусков по фактору PNL13:
	4288 или 10.166919575113809%
Количество пропусков по фактору BS1:
	0 или 0.0%
Количество пропусков по фактору BS2:
	0 или 0.0%
Количест

### Выводы

Достаточно большой датасет с большим количеством переменных (при желании можно получить больше 300 факторов). К сожалению сами по себе далеко не все факторы глубоко осмыслены, их необходимо группировать в соответствии с логикой сектора. Хорошо то, что датасет расширяем, т.к. не обезличен и то, что у нас есть дата дефолта, позволяющая получить данные о дефолте для разных срочностей.